In [1]:
import pandas as pd
import numpy as np

In [2]:
seasons = list(range(2009,2019))

# merge seasons into one complete df
df = pd.DataFrame()
for season in seasons:
    path = './data/nflscrapR-data/play_by_play_data/regular_season/reg_pbp_'+str(season)+'.csv'
    season_df = pd.read_csv(path)
    df = pd.concat([df, season_df], axis=0)
    path = './data/nflscrapR-data/play_by_play_data/post_season/post_pbp_'+str(season)+'.csv'
    post_season_df = pd.read_csv(path)
    df = pd.concat([df, post_season_df], axis=0)
print(df.shape)

print(df['play_type'].unique())

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (42,166,167,174,175,178,179,182,183,188,189,190,191,203,204,205,231,232,233,238,240,241) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (37,38,39,42,46,47,166,167,168,169,172,173,174,175,176,177,178,179,182,183,184,185,188,189,190,191,192,193,194,195,198,199,203,204,205,224,225,226,227,231,232,233,238,240,241,244,245,246,250) have mixed types. Speci

(473782, 256)
['kickoff' 'pass' 'run' 'punt' 'field_goal' 'no_play' nan 'extra_point'
 'qb_kneel' 'qb_spike']


In [3]:
# df = df.sort_values(by=['epa'],ascending=True)
# top_5 = df.head(5)
# for index, row in top_5.iterrows():
#     print("Description",row['desc'],"\n","EPA",np.round(row['epa'],1))
# df = df.sort_values(by=['epa'],ascending=False)
# top_5 = df.head(5)
# for index, row in top_5.iterrows():
#     print("Description",row['desc'],"\n","EPA",np.round(row['epa'],1))

In [4]:
print(len(df))
passes = df.loc[df.play_type=='pass']
print(len(passes))

473782
197305


In [5]:
# passes = passes.loc[passes.passer_player_name=='M.Sanu'] # for individual players
passes = passes.loc[~passes.desc.str.contains('aborted|Aborted')]
passes = passes.loc[~passes.desc.str.contains('Field Goal|Kick formation|Punt formation|Field Goal formation')]
passes = passes.loc[~passes.desc.str.contains('TWO-POINT CONVERSION ATTEMPT')]
passes = passes.loc[~passes.desc.str.contains('play under review|PENALTY')]
passes = passes.loc[(passes.wp >= 0.1)&(passes.wp <= 0.9)]
print(len(passes))


150959


In [6]:
# add column that counts how many times a player throws the ball in the dataset - RB's and WR's don't throw much obv
passes['passer_count'] = passes.groupby(['passer_player_name'])['epa'].transform("count")

In [7]:
new_pass = passes.loc[passes.passer_count <= 30]
new_pass2 = passes.loc[passes.desc.str.contains('Flea|flea')]
new_pass = pd.concat([new_pass,new_pass2],axis=0)
print(len(new_pass))

681


In [8]:
# doesn't catch everyone
print(len(new_pass))

blacklist = ["D.Fales","M.Brunell","B.Croyle","R.Smith","T.Bouman","T.Thigpen","C.Shaw","S.Mannion","C.Simms","J.O'Sullivan","Shaun.Hill","P.White","M.Simms","B.St. Pierre","R.Bartel","K.Allen","J.Palmer","L.Brown","M.Kafka","Ale.Smith","J.Losman","Matt.Moore","Dom.Davis","L.Thomas","R.Nassib","T.Boykin","C.Jones","C.Cook","J.Callahan","N.Sudfeld","R.Griffin III","K.Lauletta","G.Gilbert","J.Dobbs","J.Rudock","A.Tanney","Jo.Freeman","S.Renfree","Mat.Moore"]
new_pass = new_pass.loc[~new_pass.passer_player_name.isin(blacklist)]
new_pass = new_pass.loc[~new_pass.passer_player_name.str.contains('3rd QB')]

passes = passes.loc[~passes.passer_count > 30] # control set

print(len(new_pass))

681
260


In [9]:
for index, row in new_pass.iterrows():
    print(row['passer_player_name'], row['desc'])

M.Clayton (15:00) M.Clayton pass incomplete short right to A.Bryant.
A.Randle El (7:04) A.Randle El sacked at NYG 48 for -11 yards (sack split by C.Blackburn and J.Tuck).
C.Portis (6:40) Lateral from 17-Cambell C.Portis pass incomplete short right to C.Cooley. Penalty on WAS-W.Montgomery, Offensive Holding, declined.
M.Spurlock (2:52) Spurlock in at QB (Shotgun) M.Spurlock pass incomplete deep right to V.Davis.
M.Clayton (12:11) M.Clayton pass short left to R.Rice to SD 30 for no gain (S.Phillips).
M.Bradley (10:17) #7 M Cassel laterals to #83 M Bradley. M.Bradley pass short right to J.Charles to PHI 26 for 26 yards (A.Jordan; T.Laws).
R.Brown (13:40) Direct snap to 23 - R. Brown. (Shotgun) R.Brown pass deep left to A.Fasano to NYJ 45 for 21 yards (J.Leonhard).
R.Brown (2:30) Direct snap to 23 - R. Brown. (Shotgun) R.Brown pass incomplete short left to J.Haynos.
J.Cribbs (1:09) J.Cribbs pass incomplete short right to C.Stuckey. Coverage by #43 Polamalu.
J.Cribbs (1:05) (Shotgun) J.Crib

In [11]:
runs = df.loc[df.play_type=='run']
runs = runs.loc[~runs.desc.str.contains('PENALTY|Aborted|Penalty|TWO-POINT CONVERSION ATTEMPT')]
runs = runs.loc[~runs.desc.str.contains('RECOVERED by WAS-D.Hall at WAS 25. D.Hall for 75 yards,')] # play i noticed that wasn't a trick play
runs = runs.loc[(runs.wp >= 0.1)&(runs.wp <= 0.9)]

In [12]:
print(len(runs))
new_runs = runs.loc[runs.desc.str.contains('lateral|Lateral|reverse|Reverse')]
runs = runs.loc[~runs.desc.str.contains('lateral|Lateral|reverse|Reverse')] # remove trick plays from control dataset
print(len(new_runs))

106811
180


In [13]:
for index, row in new_runs.iterrows():
    print(row['rusher_player_name'], row['desc'])

C.Ochocinco (4:25) C.Ochocinco right end to CIN 41 for 8 yards (C.Bailey). {Double reverse, Palmer to Coles to Ochocinco}
B.Smith (8:38) B.Smith left end to NE 15 for 7 yards (S.Springs, J.Green). #6 Sanchez handed off to #29 Washington who handed off on the reverse to #16 B.Smith
T.Owens (13:20) T.Owens left end to TB 41 for 13 yards (G.Adams, B.Ruud). Reverse, handoff from F.Jackson.
R.Parrish (1:58) R.Parrish right end to BUF 36 for 9 yards (B.Ruud, W.Allen). Reverse, handoff from T.Edwards.
M.Clayton (8:13) (No Huddle, Shotgun) M.Clayton right end to CLE 28 for 12 yards (M.Adams). Clayton carries ball on pitch from Flacco on reverse.
J.Edelman (13:07) J.Edelman left end pushed ob at NE 38 for 5 yards (T.DeCoud). reverse
R.Bush (5:36) R.Bush right end to BUF 36 for 14 yards (M.Stroud). Reverse, handoff from D.Brees. R.Bush lined up at WR.
C.Johnson (6:39) #81 Johnson took the reverse handoff from #34 Smith C.Johnson left end to CHI 37 for 12 yards (D.Manning).
M.Bradley (9:14) M.Bra

In [16]:
print(new_runs.epa.mean())
print(new_runs.epa.std())

0.45801707763799593
1.222616195386599


In [15]:
print(runs.epa.mean())
print(runs.epa.std())

-0.04003251724130401
0.9955553290395552
